In [34]:
import twitter
import pandas as pd

In [35]:
key_path = './access_keys.txt'
keys = open(key_path,'r').read().splitlines()
key,key_secret,access_token,access_token_secret = keys

In [36]:
api = twitter.Api(consumer_key=key,
                  consumer_secret=key_secret,
                  access_token_key=access_token,
                  access_token_secret=access_token_secret)

In [37]:
house_members = api.GetListMembers(slug='u-s-representatives',owner_screen_name='cspan')
house_handles = [(h.name, h.screen_name) for h in house_members]

In [39]:
congress = 115
path_base = './data_US_congress/'
df = pd.read_csv(path_base + 'H' + str(congress) + '.csv')
all_members = df

In [40]:
all_members['TwitterHandle'] = 'N/A'

#Scoring function for match
f = lambda n,h,last,first: 100*(int(last in h) + int(last in n)
                            ) + 10*(int(first in h) + int(first in n)
                            ) + 1*int('Rep' in h)

for name, handle in house_handles:
    ID = None
    bestScore = 1
    for i,row in enumerate(all_members.iterrows()):
        if not row[1]['TwitterHandle'] == 'N/A':
            continue
        first,last,*_ = (i for i in row[1]['WikiPageName'].split('_') if len(i) > 1)
        
        score = f(name,handle,first,last)
        if score > bestScore:
            bestScore = score
            ID = (handle,i)
        
    if not ID == None:
        all_members['TwitterHandle'][ID[1]] = ID[0]

In [41]:
tweets = {}
for name,screen_name in house_handles:
    tweets[screen_name] = api.GetUserTimeline(screen_name=screen_name,count=200)

In [42]:
path_base = './data_US_congress_mined/'
all_members.to_csv(path_base + 'H' + str(congress) + '_tw.csv')

In [43]:
import pickle
with open(path_base + 'tweets', 'wb') as handle:
    pickle.dump(tweets, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
#with open(path_base + 'tweets', 'rb') as handle:
#    tweets = pickle.load(handle)